# Setup

In [1]:
print("Make sure you have started HEASoft and CIAO!")

Make sure you have started HEASoft and CIAO!


In [2]:
#clear session
%reset -f

import warnings

import datetime
currentDT = datetime.datetime.now()
print(currentDT.strftime("%Y-%m-%d %H:%M:%S"),"\n")

import subprocess

import os
print("working in directory:")
print(os.getcwd())

2021-12-16 16:42:50 

working in directory:
/home/cg18394/scripts


# Crop image to smaller region and smooth

In [3]:
print("This command assumes you have an image called img.fits in this directory")

This command assumes you have an image called img.fits in this directory


In [4]:
# set ra, dec and R500
ra = '8:37:52.13'
dec = '+1:29:49.99'
r500 = 100 # in arcsec
size = 2*r500

# make into a region string
reg = f'box({ra},{dec},{size}",{size}")'
print(reg)

box(8:37:52.13,+1:29:49.99,200",200")


In [5]:
# crop image
# run command and store output in variable
out = subprocess.run(['dmcopy', f'img.fits[(x,y)={reg}]', 'small.fits', 'clobber=yes'], capture_output=True, text=True)
# check command ran okay
if (out.returncode > 0): print("ERROR:",out.stderr) 


ERROR: 
Failed to open virtual file img.fits[(x,y)=box(8:37:52.13,+1:29:49.99,200",200")]
# DMCOPY (CIAO 4.13): [DM FITS kernel]: File img.fits does not exist



In [11]:
# smooth image
# run command and store output in variable
smooth = 4 # smooth by 4 pixels (just an example)
out = subprocess.run(['fgauss', 'small.fits', 'smooth.fits', f'{smooth}', 'datatype=d', 'clobber=yes'], capture_output=True, text=True)
# check command ran okay
if (out.returncode > 0): print("ERROR:",out.stderr) 


# Find peak

In [12]:
# run command and store output in variable
out = subprocess.run(['dmstat', 'smooth.fits', 'centroid=no'], capture_output=True, text=True)
# check command ran okay
if (out.returncode > 0): print("ERROR:",out.stderr) 
# get the main output from the command
out = out.stdout
print(out)

PRIMARY
    min:	0 	      @:	( 1712 1593 )
    max:	0.51475955791 	      @:	( 1695 1618 )
   mean:	0.048218380773 
  sigma:	0.077968075386 
    sum:	122.95687097 
   good:	2550 
   null:	51 



In [13]:
# split variable by lines
lines = out.splitlines()
print(lines)

['PRIMARY', '    min:\t0 \t      @:\t( 1712 1593 )', '    max:\t0.51475955791 \t      @:\t( 1695 1618 )', '   mean:\t0.048218380773 ', '  sigma:\t0.077968075386 ', '    sum:\t122.95687097 ', '   good:\t2550 ', '   null:\t51 ']


In [14]:
# get the line with "max" in it
matches = [x for x in lines if "max" in x]
print(matches)

['    max:\t0.51475955791 \t      @:\t( 1695 1618 )']


In [15]:
# split on whitespace
words = matches[0].split()
print(words)

['max:', '0.51475955791', '@:', '(', '1695', '1618', ')']


In [16]:
# get the x and y values
xpeak = words[4]
ypeak = words[5]
print("Peak is at",xpeak,ypeak)

Peak is at 1695 1618
